In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from get_iEEG_data import *
from spike_detector import *
from spike_morphology import *
from iEEG_helper_functions import *

In [ ]:
def create_pwd_file(username, password, fname=None):
    if fname is None:
        fname = "{}_ieeglogin.bin".format(username[:3])
    with open(fname, "wb") as f:
        f.write(password.encode())
    print("-- -- IEEG password file saved -- --")


create_pwd_file("dma", "mycqEv-pevfo4-roqfan")

with open("dma_ieeglogin.bin", "r") as f:
    s = Session("dma", f.read())

ds = s.open_dataset("HUP210_phaseII")
all_channel_labels = np.array(ds.get_channel_labels())
label_idxs = electrode_selection(all_channel_labels)
labels = all_channel_labels[label_idxs]

In [ ]:
ieeg_data, fs = get_iEEG_data(
    "dma",
    "dma_ieeglogin.bin",
    "HUP210_phaseII",
    (179677 + (72600 / 1024)) * 1e6,
    (179677 + (72600 / 1024) + 120) * 1e6,
    labels,
)

fs = int(fs)

In [ ]:
good_channels_res = detect_bad_channels_optimized(ieeg_data.to_numpy(), fs)
good_channel_indicies = good_channels_res[0]
good_labels = labels[good_channel_indicies]
ieeg_data = ieeg_data[good_labels]

ieeg_data = common_average_montage(ieeg_data)


# Apply the filters directly on the DataFrame
ieeg_data = pd.DataFrame(notch_filter(ieeg_data.values, 59, 61, fs))
ieeg_data = pd.DataFrame(bandpass_filter(ieeg_data.values, 1, 70, fs))

ieeg_data

In [ ]:
output = spike_detector(
    data=ieeg_data,
    fs=fs,
    labels=good_labels,
)
print(f"{len(np.unique(output[:, 2]))} spikes detected")

In [ ]:
output = output.astype(int)
output

In [ ]:
num_good = 0

for spike in output[45:]:
    channel_id = spike[1]
    peak_index = spike[0]
    spike_signal = ieeg_data[peak_index - 1000 : peak_index + 1000][
        channel_id
    ].to_numpy()

    basic_features, advanced_features, is_valid, bad_reason = extract_spike_morphology(
        spike_signal
    )

    if is_valid:
        num_good += 1
        peak, left_point, right_point, slow_end, slow_max = basic_features
        print(basic_features)
        print(advanced_features)
        plt.plot(spike_signal)
        plt.plot(peak, spike_signal[peak], "x")
        plt.plot(left_point, spike_signal[left_point], "o")
        plt.plot(right_point, spike_signal[right_point], "o")
        plt.plot(slow_end, spike_signal[slow_end], "o", color="k")
        plt.title("A spike")
        plt.xlim(250, 1750)
        plt.show()
    elif bad_reason != "Short segment":
        print(bad_reason)
        # plt.plot(spike_signal)
        # plt.title(f"NOT a spike because of {bad_reason}")
        # plt.xlim(250, 1750)
        # plt.show()